<a href="https://colab.research.google.com/github/ntkhang2003/ML-DL-learning/blob/main/Copy_of_Linear_Regression_bt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Tải dữ liệu từ kaggle lên colab

## 0.1 Tạo tài khoản trên kaggle 
Tạo tài khoản của các bạn tại đường dẫn: https://www.kaggle.com/account/login, kaggle cho phép sử dụng gmail nên các bạn hãy dùng mail trường cho dễ quản lý.

## 0.2 API Credentials:
Để tải dữ liệu từ Kaggle vào Google Colab, các bạn có thể bỏ bước trung gian là qua máy local các bạn. Kaggle hỗ trợ điều này bằng API Credentials, giúp xác thực máy ảo của Colab được phép truy cập dữ liệu từ Kaggle

- B1: Nhấn biểu tượng avatar bên phải trên cùng -> nhấn vào "Your profile".
- B2: Tại thanh bar đầu tiên, các bạn vào mục "Account".
- B3: Tìm API của Kaggle và tạo token ("kaggle.json") tại "Create new token".

Các bạn tải lên máy ảo của Colab file "kaggle.json"

Các bạn chạy các dòng lệnh bên dưới là đã xong quá trình cài kaggle

In [ ]:
!pip install kaggle 
!mkdir ~/.kaggle
%cd /content
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists
/content


## 0.3 Tải bất kì bộ dữ liệu nào

Giả sử bạn muốn tải dữ liệu từ: https://www.kaggle.com/datasets/harlfoxem/housesalesprediction

In [ ]:
### Cú pháp kaggle datasets download -d user/dataset_name -p ./folder_output/
!kaggle datasets download -d harlfoxem/housesalesprediction

  0% 0.00/780k [00:00<?, ?B/s]
100% 780k/780k [00:00<00:00, 112MB/s]


In [ ]:
!unzip housesalesprediction.zip

Archive:  housesalesprediction.zip
  inflating: kc_house_data.csv       


# 1. Bài tập linear regression với đa biến

In [ ]:
import pandas as pd
import torch

### Đọc dữ liệu huấn luyện
df_train = pd.read_csv('/content/kc_house_data.csv')

In [ ]:
df_train.isnull().values.any()

False

Các thông tin như về id sẽ không được quan trọng trong Linear Regression

In [ ]:
### Mặc định hàm drop sẽ bỏ hàng hoặc cho axis = 0, và các bạn cho index vào để bỏ hàng
### Ở đây axis = 1 tượng trưng cho cột, 

### Hàm drop ko thực sự thao tác trên dữ liệu của chúng ta
### Nên ta cần gán dữ ngược lại cho df_train
### Ngoài ra để rút gọn dấu bằng ta có thể thay 
### parameter inplace = True (rất hay gặp tại các hàm thao tác trên dữ liệu trong pandas)
### https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
df_train = df_train.drop(["id"] , axis = 1) 

In [ ]:
### Ví dụ bỏ hàng 
# df_train.head().drop([1,2,3], axis = 0)


Ngoài ra, giá của một ngôi nhà có thể không phụ thuộc vào vĩ độ (latitude) và kinh độ (longtitude) và zipcode

In [ ]:
df_train = df_train.drop(["date","lat","long","zipcode"] , axis = 1) 

In [ ]:
df_train.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,1690,7639
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,1800,7503


In [ ]:
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import train_test_split

#### Ở đây nếu không dùng scaler thì tí nữa loss sẽ ra Nan, các bạn có thể thử không dùng
#### và kiểm nghiệm lại

scaler = preprocessing.StandardScaler().fit(df_train)

data_numpy = scaler.transform(df_train)

#### Chia train test, nhưng thực ra là có dữ liệu để kiểm nghiệm cho nên mình gọi là val luôn
train, val = train_test_split(data_numpy,test_size = 0.2)

feature_names = df_train.columns

train_np = torch.tensor(train)

X_train = train_np[:,feature_names != "price"]
y_train = train_np[:,feature_names == "price"]

val_np = torch.tensor(val)

X_val = val_np[:,feature_names != "price"]
y_val = val_np[:,feature_names == "price"]

In [ ]:
#### Tạo ma trận W và bias b
#### Để dự đoán kết quả 
W = torch.tensor( torch.rand(X_train.shape[1],1), requires_grad= True)
b = torch.tensor( torch.rand(1,), requires_grad= True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [ ]:
index = torch.randperm(100)

##### dự đoán 32 mẫu giá nhà, kết quả phải có shape là [32,15]
y_predict = X_train[index[0:32],:] @ W.double() + b.double()
assert list(y_predict.shape) == [32,1], "Not matching shape"

##### dự đoán 64 mẫu giá nhà, kết quả phải có shape là [64,15]
y_predict = X_train[index[0:64],:] @ W.double() + b.double()
assert list(y_predict.shape) == [64,1], "Not matching shape"

diff = y_predict - y_train[index[0:64],:]
assert list(diff.shape) == [64,1], "Not matching shape"
loss = torch.sum(diff*diff)/(2*diff.numel()) 
loss

tensor(8.0765, dtype=torch.float64, grad_fn=<DivBackward0>)

In [ ]:
X_train.shape

torch.Size([17290, 15])

In [ ]:
import pandas as pd
import torch

### Đọc dữ liệu huấn luyện
df_train = pd.read_csv('/content/kc_house_data.csv')
df_train = df_train.drop(["id"] , axis = 1) 

df_train = df_train.drop(["date","lat","long","zipcode"] , axis = 1) 
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import train_test_split


scaler = preprocessing.StandardScaler().fit(df_train)

data_numpy = scaler.transform(df_train)

#### seed thường thấy trong numpy, pytorch, tensorflow, ...
#### Hàm này giúp mỗi lần thực hiện lần sau, bộ dữ liệu sẽ không đổi khi split
#### Giúp kết quả không phụ thuộc vào cách chia dữ liệu
#### mà là ở bộ trọng số, các hyperparameter khác nhau 
torch.manual_seed(42)

train, val = train_test_split(data_numpy,test_size = 0.2)

feature_names = df_train.columns

train_np = torch.tensor(train)

X_train = train_np[:,feature_names != "price"]
y_train = train_np[:,feature_names == "price"]

val_np = torch.tensor(val)

X_val = val_np[:,feature_names != "price"]
y_val = val_np[:,feature_names == "price"]

In [ ]:
W = torch.tensor( torch.rand(X_train.shape[1],1), requires_grad= True)
b = torch.tensor( torch.rand(1,), requires_grad= True)

variable_list = [W,b]
# for var in variable_list:
#   var.required_grad

number_of_epochs = 20
batch_size = 32
learning_rate = 1e-2


num_of_samples = X_train.shape[0]
shuffle_indices = torch.randperm(num_of_samples)

val_every = 5

for epoch in range(number_of_epochs):
  #### Train loop

  for k in (0,num_of_samples//batch_size):  
    index = shuffle_indices[k*batch_size:(k+1)*batch_size]
    
    ### Tính toán y pred : X W + b
    batch_X_train   = X_train[index,:]
    batch_y_true    = y_train[index,:]

    ############# Code tại đây #############
    batch_y_predict = batch_X_train @ W.double() + b.double()
    ########################################
    
    diff = batch_y_predict - batch_y_true

    ############# Code tại đây #############
    ### loss = tổng(diff ^ 2) / số mẫu lấy ra 
    loss = torch.sum(diff**2) / (2*batch_size)
    ########################################
    
    loss.backward()
    with torch.no_grad():
      W -= learning_rate*W.grad
      b -= learning_rate*b.grad
      W.grad.zero_()
      b.grad.zero_()
  #### Print loss
  print("Train loss {:.6f} at epoch {}".format(loss,epoch))
  #### Kiểm tra loss khi thực hiện dự đoán trên tập val mỗi 5 epoch train
  if epoch % val_every == 0:
    ### Val loop
    ### Lúc val ta không cần tính đạo hàm, vì thế cần tắt grad của các biến 
    W.required_grad = False
    b.required_grad = False
    val_loss = []
    val_shuffle_indices = torch.randperm(X_val.shape[0])

    for k in range(0,X_val.shape[0]//batch_size): 
      index = val_shuffle_indices[k*batch_size:(k+1)*batch_size]

      ### Tương tự ### Tính toán y pred : X W + b
      batch_X_val     = X_val[index,:]
      batch_y_true    = y_val[index,:]

      ############# Code tại đây #############
      batch_y_predict = batch_X_val @ W.double() + b.double()
      ########################################

      diff = batch_y_predict - batch_y_true

      ############# Code tại đây #############
      ### loss = tổng(diff ^ 2) / số mẫu lấy ra 
      loss = torch.sum(diff**2) / (2*batch_size)
      ########################################
      val_loss.append(loss)
    val_loss_avg = torch.tensor(val_loss).sum()
    val_loss_avg = val_loss_avg / len(val_loss)
    print("Average val loss {:.6f} at epoch {}".format(val_loss_avg,epoch))

    ### Sau đó, ta cần mở lại cho việc huấn luyện bằng cách
    W.required_grad = True
    b.required_grad = True

Train loss 10.187377 at epoch 0
Average val loss 6.828555 at epoch 0
Train loss 8.583275 at epoch 1
Train loss 7.244691 at epoch 2
Train loss 6.126875 at epoch 3
Train loss 5.192682 at epoch 4
Train loss 4.411271 at epoch 5
Average val loss 3.444369 at epoch 5
Train loss 3.757033 at epoch 6
Train loss 3.208697 at epoch 7
Train loss 2.748590 at epoch 8
Train loss 2.362029 at epoch 9
Train loss 2.036807 at epoch 10
Average val loss 2.038270 at epoch 10
Train loss 1.762778 at epoch 11
Train loss 1.531501 at epoch 12
Train loss 1.335955 at epoch 13
Train loss 1.170295 at epoch 14
Train loss 1.029655 at epoch 15
Average val loss 1.421110 at epoch 15
Train loss 0.909984 at epoch 16
Train loss 0.807902 at epoch 17
Train loss 0.720593 at epoch 18
Train loss 0.645708 at epoch 19


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
